First we need to import used libraries:

In [ ]:
import tensorflow as tf, pandas as pd, matplotlib.pyplot as plt, numpy as np, os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Reading the file:

In [ ]:
df = pd.read_csv(os.getcwd() + "//datasets//age_gender.csv")
# we don't need to shuffle, I just wanted to check random elements
df = shuffle(df)
df.head()

Checking if there are any null values:

In [ ]:
print(df.isnull().sum())

First split each pixel value and convert to float, only then we can normalize values of pixels from 0 - 255 to 0 - 1:

In [ ]:
df['pixels'] = df['pixels'].apply(lambda x: np.array(x.split(), dtype = "float32"))
df['pixels'] = df['pixels'].apply(lambda x: x / 255)
df.head()

Display some exemplary images:

In [ ]:
# size of the whole grid of images
plt.figure(figsize=(10, 10))

# display first 25 images in a 5x5 grid
for i in range(0, 25):
    plt.subplot(5, 5, (i % 25) + 1)
    plt.grid(False)
    #disable x and y axis description
    plt.xticks([])
    plt.yticks([])
    # each image is builds from 48 x 48 pixels
    plt.imshow(df['pixels'].iloc[i].reshape(48,48), cmap='gray')
    # could app mapping that if G = 0 then male, = 1 then female and similar with ethnicity
    # A = age, G = gender, E = ethnicity
    plt.xlabel("A: "+ str(df['age'].iloc[i])+ " G: " + str(df['gender'].iloc[i])+ " E: " + str(df['ethnicity'].iloc[i]))
plt.show()

We need to reshape pixels for convolution:

In [ ]:
x = np.array(df['pixels'].tolist())
# x.shape[0] = 23705, those are number of entries in db file, last argument is 1 if greyscale, 3 if rgb images
x = x.reshape(x.shape[0], 48, 48, 1)

Prepare y values:

In [ ]:
y_age = np.array(df['age'])
y_ethnicity = np.array(df['ethnicity'])
y_gender = np.array(df['gender'])

Split data into training and testing sets for each of the models (age, ethnicity and gender):

In [ ]:
seed = 100
x_train_age, x_test_age, y_train_age, y_test_age = train_test_split(x, y_age, test_size = 0.2, random_state = seed)
x_train_ethnicity, x_test_ethnicity, y_train_ethnicity, y_test_ethnicity = train_test_split(x, y_ethnicity, test_size = 0.2, random_state = seed)
x_train_gender, x_test_gender, y_train_gender, y_test_gender = train_test_split(x, y_gender, test_size = 0.2, random_state = seed)

## Age prediction
Build the model:

In [ ]:
age_model = Sequential()

age_model.add(Conv2D(16, kernel_size = (3, 3), strides = 1, input_shape = x.shape[1:], padding = "same", activation = 'relu'))
age_model.add(MaxPooling2D(pool_size = (2,2)))

age_model.add(Conv2D(32, kernel_size = (3, 3), strides = 1, padding = "same", activation = 'relu'))
age_model.add(MaxPooling2D(pool_size = (2,2)))

age_model.add(Conv2D(64, kernel_size = (3, 3), strides = 1, padding = "same", activation = 'relu'))
age_model.add(MaxPooling2D(pool_size = (2,2)))

age_model.add(Flatten())

age_model.add(Dense(128, activation = 'relu'))
age_model.add(Dropout(0.3))
age_model.add(Dense(1, activation = 'softmax')) # sigmoid

# check different optimizers and loss functions 'binary_crossentropy'
age_model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
# age_model.summary()

Training:

In [ ]:
age_history = age_model.fit(x_train_age, y_train_age, epochs = 20, validation_data = (x_test_age, y_test_age))

Testing:

In [ ]:
val_loss, val_acc = age_model.evaluate(x_test_age, y_test_age)

In [ ]:
acc = np.array(age_history.history['accuracy'])
val_acc = np.array(age_history.history['val_accuracy'])
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()